# Monitoramento de Vazão de Água para Prevenção de Alagamentos

Este projeto tem como objetivo apoiar a Defesa Civil na análise da vazão de água em uma área urbana sujeita a alagamentos. A função modela a taxa de entrada de água em uma bacia de contenção durante uma tempestade.

**Foi escolha a função B:**
$$f(t) = 600 \cdot e^{0.15t} \cdot \sin(0.7t)$$

Onde:

- $f(t)$ é a vazão (em litros por minuto);
- $t$ é o tempo (em minutos).

Vamos realizar análises gráficas, derivadas, integrais e uma simulação de risco.

In [1]:
# Instalando bibliotecas (pelo gerenciador de pacotes pip)
!pip install numpy matplotlib sympy scipy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\gabri\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Importando bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, sin, exp, diff, lambdify
from scipy.integrate import quad

In [ ]:
# Definindo a função simbólica e numérica
t = symbols('t')
f_sym = 600 * exp(0.15*t) * sin(0.7*t)
f_lamb = lambdify(t, f_sym, modules=['numpy'])

In [ ]:
# Função para plotagem genérica
def plot_function(func, t_vals, title, ylabel, label=None, axhline=None):
    plt.figure(figsize=(10, 6))
    plt.plot(t_vals, func(t_vals), label=label)
    if axhline:
        plt.axhline(y=axhline, color='r', linestyle='--', label='Limite Crítico')
    plt.title(title)
    plt.xlabel('Tempo (min)')
    plt.ylabel(ylabel)
    if label:
        plt.legend()
    plt.grid(True)
    plt.show()

# Definindo intervalo de tempo
t_vals = np.linspace(0, 20, 1000)

# Gráfico da vazão
plot_function(f_lamb, t_vals, 'Vazão de Água durante a Tempestade', 'Vazão (L/min)', label='f(t)')

## Derivada em $t = 10$

A derivada de $f(t)$ representa a **variação da vazão ao longo do tempo**.
Se o valor da derivada for positivo em $t = 10$, a vazão está aumentando — o que pode indicar aumento do risco. Se for negativo, a vazão está diminuindo.
O valor numérico obtido indica o ritmo dessa variação.

In [ ]:
# Cálculo da derivada em t=10
valor_derivada_t10 = f_prime_lamb(10)
print(f'Derivada em t=10: {valor_derivada_t10:.2f} L/min²')

# Interpretação do resultado
if valor_derivada_t10 > 0:
    print('A vazão está aumentando em t=10 min, indicando possível aumento do risco de alagamento.')
elif valor_derivada_t10 < 0:
    print('A vazão está diminuindo em t=10 min, sugerindo uma redução do risco de alagamento.')
else:
    print('A vazão está estável em t=10 min (taxa de variação zero).')

## Estimativa do Limite de $t$ quando $t \to +\infty$

A função tem um termo exponencial crescente, $e^{0.15t}$, e um termo oscilatório (atingindo limites de positivos e negativos absolutos), $sin(0.7t)$. Sendo assim, um limite oscilante.

**Conclusão**: O limite não existe, pois a função oscila com crescimento cada vez maior. Isso indica que a vazão pode crescer sem controle ao longo do tempo, o que representa risco de alagamento grave se não houver contenção.

In [ ]:
# Derivada simbólica e numérica
f_prime = diff(f_sym, t)
f_prime_lamb = lambdify(t, f_prime, modules=['numpy'])
valor_derivada_t10 = f_prime_lamb(10)
print(f'Derivada em t=10: {valor_derivada_t10:.2f} L/min²')

# Segunda derivada
f_second = diff(f_sym, t, 2)
f_second_lamb = lambdify(t, f_second, modules=['numpy'])

# Gráficos da derivada e segunda derivada
plot_function(f_prime_lamb, t_vals, 'Variação da Vazão', 'df/dt (L/min²)', label='f\' (t)')
plot_function(f_second_lamb, t_vals, 'Aceleração da Vazão', 'd²f/dt² (L/min³)', label='f\'\' (t)')

# Verificação de alerta baseado na derivada
if valor_derivada_t10 > 1000:
    print('Alerta: Vazão aumentando rapidamente em t=10 min!')

O valor da derivada em $t=10$ é aproximadamente $n$ L/min². Isso indica que a vazão está aumentando ou caindo a esse ritmo naquele instante.


## Volume acumulado de água (0 a 20 minutos)

O valor da integral representa o volume total de água que entrou na bacia em 20 minutos.
Se o volume ultrapassar a capacidade da bacia, ocorre alagamento.
Esse resultado pode ser comparado com uma capacidade crítica hipotética, por exemplo, 8000 L.


In [ ]:
# Cálculo da integral definida para t=20 (apenas para exibição)
integral_result, integral_error = quad(f_lamb, 0, 20)
print(f'Volume acumulado (0 a 20 min): {integral_result:.2f} L (erro estimado: {integral_error:.2e})')

# Simulação de volume acumulado e volume líquido com escoamento
taxa_escoamento = 50  # L/min
volume_acumulado = [quad(f_lamb, 0, t)[0] for t in t_vals]  # Calcula o volume acumulado para cada t
volume_liquido = [vol - taxa_escoamento * t for vol, t in zip(volume_acumulado, t_vals)]  # Ajusta com escoamento

# Gráfico do volume acumulado e líquido com escoamento
plt.figure(figsize=(10, 6))
plt.plot(t_vals, volume_acumulado, label='Volume Acumulado e Líquido')
plt.axhline(y=8000, color='r', linestyle='--', label='Limite Crítico')
plt.title('Volume Acumulado de Água e Líquido com Escoamento')
plt.xlabel('Tempo (min)')
plt.ylabel('Volume (L)')
plt.legend()
plt.grid(True)
plt.show()

# Conclusão
Foram obtidos os seguintes resultados:

- **Gráfico dinâmico**: Mostrou que a vazão cresce rapidamente com o tempo, devido ao fator exponencial, com oscilações causadas pelo seno.

- **Limite da função**: Não existe para $t \to +\infty$ devido ao crescimento exponencial e à oscilação contínua, o que indica risco crescente e instabilidade da vazão.

- **Derivada em $t=10$**: Mostrou a taxa de variação da vazão naquele instante. Um valor positivo indica que a água estava entrando com velocidade crescente naquele momento.

- **Integral definida de 0 a 20 minutos**: Representou o volume total de água acumulada no intervalo. Comparado com a capacidade da bacia (8000 L), conseguimos detectar se há risco de alagamento.